In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [4]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

In [5]:
import logging
logging.basicConfig(filename='p3ht_et.log',level=logging.DEBUG)

Now let's create project and resource to keep track of our work

In [6]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [7]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [8]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [9]:
res_local.make_dir()

In [10]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [11]:
peregrine = streamm.Resource('peregrine')

In [12]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [13]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [14]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [15]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [16]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [17]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():

    if( particle_i.symbol == 'C' ):
        particle_i.label = "C%d"%(c_cnt)
        particle_i.resname = "SCP2"
        particle_i.residue = 1

        c_cnt +=1 
    if( particle_i.symbol == 'S' ):
        particle_i.resname = "ThS"
        particle_i.residue = 2

    if( particle_i.symbol == 'H' ):
        particle_i.label = "H%d"%(h_cnt)
        particle_i.resname = "HA"
        particle_i.residue = 3

        h_cnt +=1 


Set the force-field type and guess some reasonable charges 

In [18]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
    if( particle_i.symbol == 'C' ):
        particle_i.paramkey = 'CA'
        particle_i.charge = -0.025
    if( particle_i.symbol == 'S' ):
        particle_i.paramkey = 'S'
        particle_i.charge = -0.3
    if( particle_i.symbol == 'H' ):
        particle_i.paramkey = 'HA'
        particle_i.charge = 0.1

Check molecule is neutral 

In [19]:
total_charge = 0.0
for pkey_i, particle_i  in bbTh.particles.iteritems():
    total_charge += particle_i.charge
print total_charge

-2.77555756156e-17


Optimize structure with NWChem

But let's put it in a function this time

In [20]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_i = streamm.NWChem('nw_opt_{}'.format(bb_i.tag))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.dump_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [21]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/'

In [22]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene']

Check status unit finished

In [23]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [24]:
print nwchem_i.meta['status']

finished


In [25]:
nwchem_i.analysis()

Print energies 

In [26]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.4732391, -0.3475018, -0.3290008, 0.1318747, 0.2130991, 0.2315597, 0.2416684, 0.269473, 0.2970013, 0.3198832]
-551.190758142


Check that the positions of the structure have been optimized 

In [27]:
print bbTh.positions

[[ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
 [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
 [  3.47615240e-01  -5.79042180e-01  -8.10000000e-04]
 [ -6.58844760e-01   3.61010820e-01   0.00000000e+00]
 [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
 [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
 [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
 [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
 [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]]


In [28]:
print nwchem_i.strucC.positions

[[ -1.52900788e+00  -1.96228033e+00  -8.24550000e-04]
 [ -1.87338440e-01  -1.91419006e+00  -1.07736000e-03]
 [  3.42591710e-01  -5.70383860e-01  -6.41090000e-04]
 [ -6.05111100e-01   3.80520380e-01  -3.01900000e-05]
 [ -2.25060957e+00  -3.24146360e-01   4.65300000e-05]
 [ -2.16232414e+00  -2.82023213e+00  -1.02345000e-03]
 [  4.31209940e-01  -2.78809225e+00  -1.53823000e-03]
 [  1.39112655e+00  -3.53910890e-01  -8.21500000e-04]
 [ -4.82375920e-01   1.43982288e+00   3.51820000e-04]]


Update positions with optimized geometry 

In [29]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.52900788e+00  -1.96228033e+00  -8.24550000e-04]
1 C [ -1.87338440e-01  -1.91419006e+00  -1.07736000e-03]
2 C [ 0.34259171 -0.57038386 -0.00064109]
3 C [ -6.05111100e-01   3.80520380e-01  -3.01900000e-05]
4 S [ -2.25060957e+00  -3.24146360e-01   4.65300000e-05]
5 H [ -2.16232414e+00  -2.82023213e+00  -1.02345000e-03]
6 H [  4.31209940e-01  -2.78809225e+00  -1.53823000e-03]
7 H [  1.39112655e+00  -3.53910890e-01  -8.21500000e-04]
8 H [ -4.82375920e-01   1.43982288e+00   3.51820000e-04]


Store the results in a tar ball in the storage directory 

In [30]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [31]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bb_i.tag))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem_esp.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'SCF'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    project_i.dump_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [32]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

nw_esp_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52900788      -1.96228033      -0.00082455 \n     C      -0.18733844      -1.91419006      -0.00107736 \n     C       0.34259171      -0.57038386      -0.00064109 \n     C      -0.60511110       0.38052038      -0.00003019 \n     S      -2.25060957      -0.32414636       0.00004653 \n     H      -2.16232414      -2.82023213      -0.00102345 \n     H       0.43120994      -2.78809225      -0.00153823 \n     H       1.39112655      -0.35391089      -0.00082150 \n     H      -0.48237592       1.43982288       0.00035182 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_esp_thiophene/'

Check status until finished

In [33]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene']

In [34]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [35]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status finished


In [36]:
nwchem_i.analysis()

In [37]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge
    total_charge += p.charge
print total_charge

0 C -0.085722
1 C -0.136866
2 C -0.085772
3 C -0.108841
4 S -0.11776
5 H 0.153759
6 H 0.119036
7 H 0.109131
8 H 0.153035
0.0


Update the charges of the Buildingblock

In [38]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.085722
1 C -0.136866
2 C -0.085772
3 C -0.108841
4 S -0.11776
5 H 0.153759
6 H 0.119036
7 H 0.109131
8 H 0.153035


Store the results 

In [39]:
nwchem_i.store()

In [40]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [41]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

Store a pickled object of the Buildingblock

In [42]:
os.chdir(res_local.dir['materials']) 

In [ ]:
bbTh.dump_pickle()

Let us optimize the structure with the oplsaa force-field to check the parameters 

In [95]:
os.chdir(res_local.dir['home']) 

In [96]:
from pathlib2 import Path

In [97]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

In [98]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [99]:
print oplsaa.unit_conf['energy']

kCalmol


We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [100]:
import streamm.forcefields.particletype as particletype

In [101]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [102]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [103]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [104]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [105]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [106]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [107]:
print CA,HA

 CA epsilon:0.07 sigma:3.55  HA epsilon:0.03 sigma:2.42


In [108]:
S = particletype.Particletype('S')

In [109]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [110]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [111]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [112]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [113]:
import streamm.forcefields.bondtype as bondtype

In [114]:
bt_i = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.080,367.0)
oplsaa.add_bondtype(bt_i)

In [115]:
bt_i = bondtype.Bondtype('CA','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.400,469.0)
oplsaa.add_bondtype(bt_i)

In [116]:
bt_i = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.71,250.0)
oplsaa.add_bondtype(bt_i)

In [117]:
for btk,bt in oplsaa.bondtypes.iteritems():
    print btk,bt

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
1  bond  CT - CT type harmonic 
  harmonic r_0 = 1.530000 K = 268.000000 lammps index 0  gromacs index 0  
2  bond  CA - HA type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
3  bond  CA - CA type harmonic 
  harmonic r_0 = 1.400000 K = 469.000000 lammps index 0  gromacs index 0  
4  bond  S - CA type harmonic 
  harmonic r_0 = 1.710000 K = 250.000000 lammps index 0  gromacs index 0  


In [118]:
import streamm.forcefields.angletype as angletype

In [119]:
bat_i = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,63.0)
oplsaa.add_angletype(bat_i)

In [120]:
bat_i = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [121]:
bat_i = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(92.2,70.0)
oplsaa.add_angletype(bat_i)

In [122]:
bat_i = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [123]:
bat_i = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(111.0,70.0)
oplsaa.add_angletype(bat_i)

In [124]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
2  angle  CA - CA - CA type harmonic 
  harmonic theta_0 = 120.000000 K = 63.000000 lammps index 0  gromacs index 0  
3  angle  CA - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
4  angle  CA - S - CA type harmonic 
  harmonic theta_0 = 92.200000 K = 70.000000 lammps index 0  gromacs index 0  
5  angle  S - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
6  angle  S - CA - CA type harmonic 
  harmonic theta_0 = 111.000000 K = 70.000000 lammps index 0  gromacs index 0  


Set some reasonable dihedral parameters

In [125]:
import streamm.forcefields.dihtype as dihtype

In [126]:
# oplsaa.dihtypes = {}

In [127]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [128]:
dih_i = dihtype.Dihtype('X','S','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,2.416710,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [129]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [130]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - S - CA - X type opls 
  k1 = 0.000000 k2 = 2.416710 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 


Let us make an MD simulation of just the monomer to check that our parameters are okay

In [131]:
def lmp_run(project_i,bb_i,param_i,res_i,md_type = 'min'):
    # Create LAMMPS calculation object 
    lmp_i = streamm.LAMMPS('lmp_{}_{}'.format(md_type,bb_i.tag))
    # Set parameter container 
    lmp_i.paramC = param_i
    lmp_i.set_strucC(bb_i)
    # Set force-field parameters 
    lmp_i.set_ffparam()
    # Set resource to local
    lmp_i.set_resource(res_i)
    # Make local directories
    lmp_i.make_dir()
    # Set pbc's to on
    lmp_i.strucC.lat.pbcs = [True,True,True]
    # Change to launch directory
    os.chdir(lmp_i.dir['launch'])
    # Copy over the templates from the template directory 
    lmp_i.cp_file('templates','in',"lammps_{}.in".format(md_type),'templates','launch')
    lmp_i.cp_file('templates','run',"lammps_peregrine.pbs",'templates','launch')
    # Change to scratch
    os.chdir(lmp_i.dir['launch'])
    # Read in template files and store them as strings in the `str` dictionary
    lmp_i.load_str('templates','in')
    lmp_i.load_str('templates','run')
    # Write LAMMPS .data file
    lmp_i.write_data()
    # Replace keys in template string with properties 
    lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))
    # Add the input file to the properties to be written into the run file
    lmp_i.properties['input_in'] = lmp_i.files['input']['in']
    lmp_i.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_i.tag))
    # Save json file in root directory
    os.chdir(lmp_i.dir['home'])
    lmp_i.dump_json()
    # Run bash script or submit to cluster
    lmp_i.add_file('output','log',"%s.log"%(lmp_i.tag))
    # Save details in .json files 
    os.chdir(lmp_i.dir['home'])
    project_i.dump_json()
    #
    os.chdir(lmp_i.dir['launch'])
    lmp_i.push()
    lmp_i.run()
    # Add calculation to project
    project_i.add_calc(lmp_i,deepcopy = True)
    # 
    return project_i     

In [132]:
p3ht_et = lmp_run(p3ht_et,bbTh,oplsaa,peregrine)

In [134]:
lmp_i = p3ht_et.calculations['lmp_min_thiophene']

In [ ]:
import time

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    

In [83]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '2629.456', '2629.456', '-0.42232458', '1000000', '100', '100', '100', '7.4306668', '2617.6436', '4.9478895e-08', '0', '-0.0096913433', '6.497175', '-2.1056982', '0']
1 9823u  [0.0]
> col  1 ['29', '0', '60.985782', '60.985782', '-0.12011223', '1000000', '100', '100', '100', '1.5768279', '23.057887', '16.52658', '0', '9.1574079', '12.777921', '-2.1108416', '0']
2 9823u  [0.0, 29.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [84]:
lmp_i.store()

Read in data file positions

In [85]:
lmp_i.pull()

Traceback (most recent call last):
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 465, in format
    record.message = record.getMessage()
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 329, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Logged from file calculation.py, line 533
Traceback (most recent call last):
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/Users/tkemper/anaconda2/lib/python2.7/logging/__init__.py", line 465, in format
    record.mes

Read in data file output and update positions

In [86]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_thiophene/


In [87]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [88]:
lmp_i.read_data_pos(datafn)

0 [-0.8397361944928899, -1.8604259320930354, 0.7103947370445873]
1 [-0.39939429939118576, -1.514616676557732, -0.5674200125060925]
2 [-0.08345742660422979, -0.719127202711615, 0.5650544657029544]
3 [-0.17189308303922596, -0.16585969077160317, -0.7125743150733305]
4 [-1.7368127118826726, -0.5247837613410571, 0.002349339697659885]
5 [-1.3280873035399758, -1.6451483524595378, -0.24249693255747207]
6 [-0.08031494745141593, -1.6784404944781284, 0.46534677850210127]
7 [0.2592739005622196, -0.8171159077849899, -0.4684030587642232]
8 [-0.6714159341606227, 0.012626018197701063, 0.24219299795381527]


In [89]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [90]:
lmp_i.strucC.write_xyz()

In [91]:
bbTh.write_xyz('bbTh.xyz')

Build hexane

In [ ]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [ ]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [ ]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [ ]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

In [ ]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

In [ ]:
total_charge = 0.0
for pkey_i, particle_i  in bbHex.particles.iteritems():
            total_charge += particle_i.charge 

Check that the molecule is neutral 

In [ ]:
print total_charge

Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [ ]:
p3ht_et = nw_opt(p3ht_et,bbHex,peregrine)

In [ ]:
nwchem_i = p3ht_et.calculations['nw_opt_hexane']

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [ ]:
p3ht_et.check()

Get the calculation from the project object 

In [ ]:
nwchem_i.analysis()

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Check that the positions of the structure have been optimized 

In [ ]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

In [ ]:
print nwchem_i.strucC.positions

Update positions in Buildingblock object

In [ ]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

Store the results in a tar ball in the storage directory 

In [ ]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [ ]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

Check status unit finished

In [ ]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [ ]:
p3ht_et.check()

In [ ]:
nwchem_i.analysis()

In [ ]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

In [ ]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    total_charge += p.charge
print total_charge

Print energies 

In [ ]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

Update the charges of the Buildingblock

In [ ]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

Store the results 

In [ ]:
nwchem_i.store()

First we need to identify the bonding within the Buildingblock

In [ ]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [ ]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

Now we have optimized geometries and ESP Charges for each buildingblock

So let us make some P3HT oligomers 

In [ ]:
res_local.make_dir()

In [ ]:
os.chdir(res_local.dir['materials']) 

In [ ]:
bbTh.find_rsites()
bbHex.find_rsites()

In [ ]:
print(bbTh.show_rsites())

In [ ]:
print(bbHex.show_rsites())

In [ ]:
import streamm.structures.buildingblock as bb

In [ ]:
ht = bb.attach(bbTh,bbHex,'funccap',0,'rgcap',0,tag='3-hexyl-thiophene')

Check that the molecule looks good

In [ ]:
ht.write_xyz()

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [ ]:
ht.calc_charge()
ht.charge

In [ ]:
print(ht.show_rsites())

Now that we have the monomer let us set up the force field parameters 

Our potential is in kCal/mol (`kCalmol`) so let's get the unit dictionary and create our own default

Like the force-field example we will use the oplsaa force-field

In [ ]:
os.chdir(res_local.dir['home']) 

In [ ]:
from pathlib2 import Path

In [ ]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

In [ ]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [ ]:
print oplsaa.unit_conf['energy']

We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [ ]:
import streamm.forcefields.particletype as particletype

In [ ]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [ ]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [ ]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [ ]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [ ]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [ ]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [ ]:
print CA,HA

In [ ]:
S = particletype.Particletype('S')

In [ ]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [ ]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [ ]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [ ]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [ ]:
import streamm.forcefields.bondtype as bondtype

In [ ]:
bt_i = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.080,367.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
bt_i = bondtype.Bondtype('CA','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.400,469.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
bt_i = bondtype.Bondtype('CT','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.51,317.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
bt_i = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.71,250.0)
oplsaa.add_bondtype(bt_i)

In [ ]:
for btk,bt in oplsaa.bondtypes.iteritems():
    print btk,bt

In [ ]:
import streamm.forcefields.angletype as angletype

In [ ]:
CA_CA_CA = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
CA_CA_HA = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
CA_S_CA = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
S_CA_HA = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
S_CA_CA = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
CA_CA_CT = angletype.Angletype('CA','CA','CT',unit_conf=oplsaa.unit_conf)
HA_CA_CT = angletype.Angletype('HA','CA','CT',unit_conf=oplsaa.unit_conf)
CA_CT_HC = angletype.Angletype('CA','CT','HC',unit_conf=oplsaa.unit_conf)
CA_CT_CT = angletype.Angletype('CA','CT','CT',unit_conf=oplsaa.unit_conf)
CT_CT_HC = angletype.Angletype('CT','CT','HA',unit_conf=oplsaa.unit_conf)

In [ ]:
CA_CA_CA.setharmonic(63.0,120.0)
CA_CA_HA.setharmonic(35.0,120.0)
CA_S_CA.setharmonic(70.0,92.2)
S_CA_CA.setharmonic(70.0,111.0)
S_CA_HA.setharmonic(35.0,120.0)
CA_CA_CT.setharmonic(70.0,120.0)
HA_CA_CT.setharmonic(35.0,120.0)
CA_CT_HC.setharmonic(50.0,109.5)
CA_CT_CT.setharmonic(63.0,114.0)
CT_CT_HC.setharmonic(50.0,109.5)

In [ ]:
oplsaa.add_angletype(CA_CA_CA)
oplsaa.add_angletype(CA_CA_HA)
oplsaa.add_angletype(CA_S_CA)
oplsaa.add_angletype(S_CA_CA)
oplsaa.add_angletype(S_CA_HA)
oplsaa.add_angletype(CA_CA_CT)
oplsaa.add_angletype(HA_CA_CT)
oplsaa.add_angletype(CA_CT_HC)
oplsaa.add_angletype(CA_CT_CT)
oplsaa.add_angletype(CT_CT_HC)

In [ ]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(40.0,109.50)
oplsaa.add_angletype(bat_i)



In [ ]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

Set some reasonable dihedral parameters

Note: The inter-ring torsional is not consider as a seperate set of parameters for the simplicity of this example

In [ ]:
import streamm.forcefields.dihtype as dihtype

In [ ]:
oplsaa.dihtypes = {}

In [ ]:
dih_i = dihtype.Dihtype('CT','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('HC','CT','CT','HC',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('X','S','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,2.416710,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [ ]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

Let us make an MD simulation of just the monomer to check that our parameters are okay

In [ ]:
def lmp_run(project_i,bb_i,param_i,res_i):
    # Create LAMMPS calculation object 
    lmp_i = streamm.LAMMPS('lmp_sp_{}'.format(bb_i.tag))
    # Set parameter container 
    lmp_i.paramC = param_i
    lmp_i.set_strucC(bb_i)
    # Set force-field parameters 
    lmp_i.set_ffparam()
    # Set resource to local
    lmp_i.set_resource(res_i)
    # Make local directories
    lmp_i.make_dir()
    # Set pbc's to on
    lmp_i.strucC.lat.pbcs = [True,True,True]
    # Change to launch directory
    os.chdir(lmp_i.dir['launch'])
    # Copy over the templates from the template directory 
    lmp_i.cp_file('templates','in',"lammps_sp.in",'templates','launch')
    lmp_i.cp_file('templates','run',"lammps_peregrine.pbs",'templates','launch')
    # Change to scratch
    os.chdir(lmp_i.dir['launch'])
    # Read in template files and store them as strings in the `str` dictionary
    lmp_i.load_str('templates','in')
    lmp_i.load_str('templates','run')
    # Write LAMMPS .data file
    lmp_i.write_data()
    # Replace keys in template string with properties 
    lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))
    # Add the input file to the properties to be written into the run file
    lmp_i.properties['input_in'] = lmp_i.files['input']['in']
    lmp_i.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_i.tag))
    # Save json file in root directory
    os.chdir(lmp_i.dir['home'])
    lmp_i.dump_json()
    # Run bash script or submit to cluster
    lmp_i.add_file('output','log',"%s.log"%(lmp_i.tag))
    # Save details in .json files 
    os.chdir(lmp_i.dir['home'])
    project_i.dump_json()
    #
    os.chdir(lmp_i.dir['launch'])
    lmp_i.push()
    lmp_i.run()
    # Add calculation to project
    project_i.add_calc(lmp_i,deepcopy = True)
    # 
    return project_i     

In [ ]:
p3ht_et = lmp_run(p3ht_et,ht,oplsaa,peregrine)

In [ ]:
p3ht_et.check()

In [ ]:
lmp_i = p3ht_et.calculations['lmp_sp_3-hexyl-thiophene']

In [ ]:
lmp_i.analysis()

Energy decreased and nothing exploded so that's good

In [ ]:
lmp_i.store()

Read in data file positions

In [ ]:
lmp_i.pull()

Read in data file output and update positions

In [ ]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

In [ ]:
datafn = lmp_i.files['output']['data_1']
print datafn

In [ ]:
lmp_i.read_data_pos(datafn)

In [ ]:
print lmp_i.strucC.lat.matrix

In [ ]:
groupset_i = streamm.Groups('mol',lmp_i.strucC)
groupset_i.group_prop('mol','group_mol')

In [ ]:
groupset_i.group_pbcs()

In [ ]:
lmp_i.strucC.positions

In [ ]:
lmp_i.strucC.write_xyz()

In [ ]:
ht.write_xyz('ht.xyz')

In [ ]:
print lmp_i.strucC.lat.pbcs

Okay we have the monomer, so let's make a pentamer 

In [ ]:
penta_ht = copy.deepcopy(ht)

In [ ]:
# penta_ht = ht.prepattach('termcap',0,dir=-1,yangle=90.0)

In [ ]:
for n in range(4):
    penta_ht = bb.attach(penta_ht,ht,'termcap',1,'termcap',0,tag='penta_3-hexyl-thiophene')

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [ ]:
penta_ht.calc_charge()
penta_ht.charge

In [ ]:
penta_ht.write_xyz()

Well it's cis, but we can run some high temperature MD to randomize that 

In [ ]:
# del oplsaa.dihtypes[3]

In [ ]:
oplsaa_unit_conf = copy.deepcopy(CT.unit_conf)

In [ ]:
oplsaa_unit_conf['energy'] = 'kCalmol'

In [ ]:
pprint(oplsaa_unit_conf)

Sweet as bro!